# For experiment

In [2]:
!pip install fredapi

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import preprocessor as pre
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import numpy as np
import matplotlib.pyplot as plt
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error

In [38]:
gas_df = pre.get_Fred_data('PNGASEUUSDM',2015,2023)
wheat_df = pre.get_Fred_data('PWHEAMTUSDM',2015,2023)
ammonia_df = pre.get_Fred_data('WPU0652013A',2015,2023)

price_evo_df = pre.clean_pred_price_evo_csv('Data_flat_files/Dataset_Future_Predicting_Price_Evolutions.csv',2016,2023)

dummy_df = pre.get_dummies_and_average_price(price_evo_df,'acid',\
                                         'RM01/0001','RM01/0004','RM01/0006','RM01/0007')

acid_df = pre.generate_features(1,12,dummy_df,\
                                PNGASEUUSDM=gas_df,\
                                PWHEAMTUSDM=wheat_df,\
                                WPU0652013A=ammonia_df
                               )

print(acid_df.info())
print(acid_df.groupby(['Year','Month']).size().to_string())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 362 entries, 0 to 361
Data columns (total 44 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   RM01/0004          362 non-null    uint64        
 1   RM01/0006          362 non-null    uint64        
 2   RM01/0007          362 non-null    uint64        
 3   Time               362 non-null    datetime64[ns]
 4   Group Description  362 non-null    object        
 5   Year               362 non-null    float64       
 6   Month              362 non-null    float64       
 7   Average_price      362 non-null    float64       
 8   PNGASEUUSDM_1      362 non-null    float64       
 9   PWHEAMTUSDM_1      362 non-null    float64       
 10  WPU0652013A_1      362 non-null    float64       
 11  PNGASEUUSDM_2      362 non-null    float64       
 12  PWHEAMTUSDM_2      362 non-null    float64       
 13  WPU0652013A_2      362 non-null    float64       
 14  PNGASEUUSD

In [54]:
test = pd.read_excel('Data_flat_files/acid_df.xlsx')
test['Year'] = [date.split('-')[0] for date in test['acid_ym'].values]
test['Month'] = [date.split('-')[1] for date in test['acid_ym'].values]
test = test.drop(['Unnamed: 0','acid_ym'], axis=1)
test.rename(columns={'cat_04':'RM01/0004',\
                     'cat_06':'RM01/0006',\
                     'cat_07':'RM01/0007',\
                     'acid_price':'Average_price'},\
           inplace=True)
print(test.info())




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357 entries, 0 to 356
Data columns (total 54 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Average_price     357 non-null    float64
 1   RM01/0004         357 non-null    int64  
 2   RM01/0006         357 non-null    int64  
 3   RM01/0007         357 non-null    int64  
 4   ele_price_12      357 non-null    float64
 5   ele_price_11      357 non-null    float64
 6   ele_price_10      357 non-null    float64
 7   ele_price_9       357 non-null    float64
 8   ele_price_8       357 non-null    float64
 9   ele_price_7       357 non-null    float64
 10  ele_price_6       357 non-null    float64
 11  ele_price_5       357 non-null    float64
 12  ele_price_4       357 non-null    float64
 13  ele_price_3       357 non-null    float64
 14  ele_price_2       357 non-null    float64
 15  ele_price_1       357 non-null    float64
 16  gas_price_12      357 non-null    float64
 1

In [86]:
filter_1 = ((barry['RM01/0004'] == 0) &\
          (barry['RM01/0006'] == 0) &\
          (barry['RM01/0007'] == 0))

filter_2 = ((xiao['RM01/0004'] == 0) &\
          (xiao['RM01/0006'] == 0) &\
          (xiao['RM01/0007'] == 0))

xiao = (test[test['Year'] > "2015"])
barry = (acid_df[acid_df['Time'] < '2023-11-01'])

# print(xiao.groupby(["Year"]).size())
# print(barry.groupby(['Year']).size())
print(barry[filter_1].groupby(['Year'])['Average_price'].mean())
print(xiao[filter_2].groupby(['Year'])['Average_price'].mean())

Year
2016.0    0.492931
2017.0    0.468417
2018.0    0.676917
2019.0    0.573833
2020.0    0.482167
2021.0    1.001417
2022.0    1.295218
2023.0    0.843000
Name: Average_price, dtype: float64
Year
2016    0.491345
2017    0.470101
2018    0.676925
2019    0.564103
2020    0.483143
2021    0.988709
2022    1.285050
2023    0.850697
Name: Average_price, dtype: float64
